<a href="https://colab.research.google.com/github/danbom/AIrang/blob/eunyoung/NarrativeKoGPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NarrativeKoGPT2 학습

## 1.Google Drive 연동
- 모델 파일과 학습 데이터가 저장 되어있는 구글 드라이브의 디렉토리와 Colab을 연동.  
- 좌측상단 메뉴에서 런타임-> 런타임 유형 변경 -> 하드웨어 가속기 -> GPU 선택 후 저장

### 1.1 GPU 연동 확인

In [ ]:
!nvidia-smi

Thu Jan  7 00:41:27 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### 1.2 Google Drive 연동
아래 코드를 실행후 나오는 URL을 클릭하여 나오는 인증 코드 입력

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls

drive  sample_data


**Colab 디렉토리 아래 NarrativeKoGPT2 경로 확인**

In [ ]:
!ls drive/'My Drive'/'Colab Notebooks'/

NarrativeKoGPT2


**필요 패키지들 설치**

In [ ]:
!pip install -r drive/'My Drive'/'Colab Notebooks'/NarrativeKoGPT2/requirements.txt

     |████████████████████████████████| 348kB 4.3MB/s 
     |████████████████████████████████| 55.0MB 57kB/s 
     |████████████████████████████████| 1.1MB 41.0MB/s 
     |████████████████████████████████| 481kB 35.7MB/s 
     |████████████████████████████████| 317kB 44.7MB/s 
     |████████████████████████████████| 3.1MB 45.1MB/s 
     |████████████████████████████████| 102kB 11.0MB/s 
     |████████████████████████████████| 890kB 38.0MB/s 
     |████████████████████████████████| 7.2MB 44.3MB/s 
     |████████████████████████████████| 71kB 8.0MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp36-cp36m-linux_x86_64.whl size=588512 sha256=95875c549d4ecbb11395d89c2e78242efc748f485296fcf306f447dcf6120d0c
  Stored in directory: /root/.cache/pip/wheels/37/65/52/63032864a0f31a08b9a88569f803b5bafac8abd207fd7f7534
  Created wheel for boto3: filename=boto3-1.16.50-py2.py3-none-any.whl size=128710 sha256=5b91f47b40aa5ed6538e15d88cb7ded1ef43b71ea8bc33bcb2452c1d050bd9dc
  Stored in dir

In [ ]:
import os

import sys
sys.path.append('drive/My Drive/Colab Notebooks/')
print(os.getcwd())


/content


## 2.KoGPT2 Transfer Learning

### 2.1.Import Package

In [ ]:
import random
import torch
from torch.utils.data import DataLoader # 데이터로더
from gluonnlp.data import SentencepieceTokenizer 
from NarrativeKoGPT2.kogpt2.utils import get_tokenizer
from NarrativeKoGPT2.kogpt2.utils import download, tokenizer
from NarrativeKoGPT2.model.torch_gpt2 import GPT2Config, GPT2LMHeadModel
from NarrativeKoGPT2.util.data import NovelDataset
import gluonnlp
from tqdm import tqdm

**torch GPU 확인**

In [ ]:
print(torch.cuda.device_count())  # GPU deviec count check

1


### 2.2. koGPT-2 Config

In [ ]:
ctx= 'cuda'#'cuda' #'cpu' #학습 Device CPU or GPU. colab의 경우 GPU 사용
cachedir='~/kogpt2/' # KoGPT-2 모델 다운로드 경로
epoch =200  # 학습 epoch
save_path = 'drive/My Drive/Colab Notebooks/NarrativeKoGPT2/checkpoint/'
#use_cuda = True # Colab내 GPU 사용을 위한 값

pytorch_kogpt2 = {
    'url':
    'https://kobert.blob.core.windows.net/models/kogpt2/pytorch/pytorch_kogpt2_676e9bcfa7.params',
    'fname': 'pytorch_kogpt2_676e9bcfa7.params',
    'chksum': '676e9bcfa7'
}
kogpt2_config = {
    "initializer_range": 0.02,
    "layer_norm_epsilon": 1e-05,
    "n_ctx": 1024,
    "n_embd": 768,
    "n_head": 12,
    "n_layer": 12,
    "n_positions": 1024,
    "vocab_size": 50000
}

### 2.3 Model and Vocab Download

In [ ]:
# download model
model_info = pytorch_kogpt2
model_path = download(model_info['url'],
                       model_info['fname'],
                       model_info['chksum'],
                       cachedir=cachedir)
# download vocab
vocab_info = tokenizer
vocab_path = download(vocab_info['url'],
                       vocab_info['fname'],
                       vocab_info['chksum'],
                       cachedir=cachedir)

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


### 2.4.KoGPT-2 Model Vocab

In [ ]:
# KoGPT-2 언어 모델 학습을 위한 GPT2LMHeadModel 선언
kogpt2model = GPT2LMHeadModel(config=GPT2Config.from_dict(kogpt2_config))
# model_path로부터 다운로드 받은 내용을 load_state_dict으로 업로드
kogpt2model.load_state_dict(torch.load(model_path))

device = torch.device(ctx)
kogpt2model.to(device)

# kogpt2model.eval()
# 추가로 학습하기 위해 .train() 사용
kogpt2model.train()
vocab_b_obj = gluonnlp.vocab.BERTVocab.from_sentencepiece(vocab_path,
                                                     mask_token=None,
                                                     sep_token=None,
                                                     cls_token=None,
                                                     unknown_token=None,
                                                     padding_token='<pad>',
                                                     bos_token=None,
                                                     eos_token=None)


### 2.5. Get Batch Data using DataLoader

In [ ]:
tok_path = get_tokenizer()
model, vocab = kogpt2model, vocab_b_obj
sentencepieceTokenizer = SentencepieceTokenizer(tok_path)

#os.chdir("../")
data_file_path = 'drive/My Drive/Colab Notebooks/NarrativeKoGPT2/data/bm_novel_1/prerpcessed_bm_novel_utf8_3.txt'
batch_size = 2
novel_dataset = NovelDataset(data_file_path, vocab, sentencepieceTokenizer)
#novel_data_loader = DataLoader(novel_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
novel_data_loader = DataLoader(novel_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, collate_fn=lambda x: x)

using cached model


### 2.6. Learning rate, Loss function, Adam Optimizer

In [ ]:
learning_rate = 1e-5
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [ ]:
import subprocess

def get_gpu_memory_map():
    """Get the current gpu usage.

    Returns
    -------
    usage: dict
        Keys are device ids as integers.
        Values are memory usage as integers in MB.
    """
    result = subprocess.check_output(
        [
            'nvidia-smi', '--query-gpu=memory.used',
            '--format=csv,nounits,noheader'
        ], encoding='utf-8')
    # Convert lines into a dictionary
    gpu_memory = [int(x) for x in result.strip().split('\n')]
    gpu_memory_map = dict(zip(range(len(gpu_memory)), gpu_memory))
    return gpu_memory_map

### 2.7. KoGPT-2 Transfer Laerning

In [ ]:
# from tensorboardX import SummaryWriter
from torch.utils.tensorboard import SummaryWriter
from NarrativeKoGPT2.sampling import sample_sequence
summary = SummaryWriter()
avg_loss = (0.0, 0.0)

print('KoGPT-2 Transfer Learning Start')
epoch=50

for epoch in range(epoch):
  count = 0
  for data in novel_data_loader:
    # data = datas[0] ### lyric version 2에 있는 코드
    optimizer.zero_grad()

    if (type(data) != torch.Tensor):
      while (len(data[0]) != len(data[1])):
        if (len(data[0]) > len(data[1])):
          data[1].append(1)
        else :
          data[0].append(1)
      # data = torch.tensor(data)
      data[0] = torch.tensor(data[0])
      data[1] = torch.tensor(data[1])

    data = torch.stack(data) # list of Tensor로 구성되어 있기 때문에 list를 stack을 통해 변환해준다.

    data= data.transpose(1,0)
    data= data.to(ctx)
    model = model.to(ctx)
    
    outputs = model(data, labels=data)
    loss, logits = outputs[:2]

    avg_loss = (avg_loss[0] * 0.99 + loss, avg_loss[1] * 0.99 + 1.0)
    loss *= datas[2][0] # 특별 socre 부분
    
    loss = loss.to(ctx)
    loss.backward()
    
    optimizer.step()

    if count %100 ==0:
        print('epoch no.{} train no.{}  loss = {}   avg_loss = {}' . format(epoch, count+1, loss, avg_loss[0] / avg_loss[1]))
        # torch.save(model,save_path+'checkpoint_{}_{}.tar'.format(epoch,count))
        summary.add_scalar('loss/avg_loss', avg_loss[0] / avg_loss[1], count)
        summary.add_scalar('loss/loss', loss, count)
   
    # generator 진행
    if (count > 0 and count % 500 == 0) or (len(data) < batch_size):
        sent = sample_sequence(model.to("cpu"), sentencepieceTokenizer, vocab, sent="우리", text_size=100, temperature=0.7, top_p=0.9, top_k=100)
        sent = sent.replace("<unused0>", "\n")
        sent = sent.replace("//", "\n")
        print(sent)

        summary.add_text('Text', sent, count)

        if count > 500000:
            now = [int(n) for n in os.listdir(samples)]
            now = max(now)
            f = open(samples + str(now + 1), 'w', encoding="utf-8")
            f.write(sent)
            f.close()
      # 추론 및 학습 재개를 위한 일반 체크포인트 저장하기
    if (count >0 and count%100==0) or (len(data) < batch_size):
      torch.save({
        'epoch': epoch,
        'train_no': count,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss':loss
      }, save_path+'narrativeKoGPT2_checkpoint.tar')

    count += 1

KoGPT-2 Transfer Learning Start
epoch no.0 train no.1  loss = 6.2840776443481445
epoch no.0 train no.101  loss = 7.443725109100342
epoch no.0 train no.201  loss = 3.8746390342712402
epoch no.0 train no.301  loss = 4.118159770965576
epoch no.0 train no.401  loss = 3.7979559898376465
epoch no.0 train no.501  loss = 4.72245454788208
to_tokens: ['</s>', '<unk>', '</s>', '▁']
우리</s>

epoch no.0 train no.601  loss = 3.611882209777832
epoch no.0 train no.701  loss = 4.238971710205078
epoch no.0 train no.801  loss = 3.700378894805908
epoch no.1 train no.1  loss = 4.582493305206299
epoch no.1 train no.101  loss = 4.4023051261901855
epoch no.1 train no.201  loss = 3.47839617729187
epoch no.1 train no.301  loss = 5.305163860321045
epoch no.1 train no.401  loss = 4.164834499359131
epoch no.1 train no.501  loss = 4.615095138549805
to_tokens: ['<s>', '<unk>', '</s>', '</s>']
우리</s>

epoch no.1 train no.601  loss = 4.739813327789307
epoch no.1 train no.701  loss = 4.9747090339660645
epoch no.1 train 

In [ ]:
summary.flush()

In [ ]:
summary.close()

In [ ]:
!pip install tensorboard --upgrade

Requirement already up-to-date: tensorboard in /usr/local/lib/python3.6/dist-packages (2.4.0)


In [ ]:
!tensorboard dev upload --logdir runs \
--name "My latest experiment" \
--description "Simple comparison of several hyperparameters"

2021-01-06 07:49:40.219446: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1

***** TensorBoard Uploader *****

This will upload your TensorBoard logs to https://tensorboard.dev/ from
the following directory:

runs

This TensorBoard will be visible to everyone. Do not upload sensitive
data.

Your use of this service is subject to Google's Terms of Service
<https://policies.google.com/terms> and Privacy Policy
<https://policies.google.com/privacy>, and TensorBoard.dev's Terms of Service
<https://tensorboard.dev/policy/terms/>.

This notice will not be shown again while you are logged into the uploader.
To log out, run `tensorboard dev auth revoke`.

Continue? (yes/NO) yes

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=373649185512-8v619h5kft38l4456nm2dj4ubeqsrvh6.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scop